Esse Notebook será dedicado a experimentar como os modelos classicos aprendem com os dados youtube e spotify.

* Primeiro será feito um teste com os modelos sem nenhum ajuste previo para entender como tão prevendo os dados em primeiro momento.

* Em segundo momento será testado como a idade dos dados ta afetenado previsão do modelo em 3 diferentes faixas de tempo.

possiveis melhorias:
*   tirar os outliers de engajamento
*   utilizar tecnicas como SMOTE para aumentar o numero de musicas com ate 30 dias
*   tirar outliers

Descobertas:
* Utilizar apenas musicas recentes não garante que vamos ter resultados melhores, portanto pode usar todas as musicas
* Musicas com mais dias são mais faceis de prever
* Em primeiro momento tirar o numero de streams ajuda na previsão do primreiro mes

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [55]:
from sklearn.linear_model import  LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb

# Tratamento dos dados

In [105]:
spotify_youtube_pt1 = pd.read_csv("Dados/spotify_youtube_up_pt1.csv")
spotify_youtube_pt2 = pd.read_csv("Dados/spotify_youtube_up_pt2.csv")

In [106]:
data_extracao = pd.to_datetime("2023-02-07")
spotify_youtube_pt1["upload_date"] = pd.to_datetime(spotify_youtube_pt1["upload_date"])
spotify_youtube_pt1["dias_na_plataforma"] = (data_extracao - spotify_youtube_pt1["upload_date"]).dt.days

spotify_youtube_pt2["upload_date"] = pd.to_datetime(spotify_youtube_pt2["upload_date"])
spotify_youtube_pt2["dias_na_plataforma"] = (data_extracao - spotify_youtube_pt2["upload_date"]).dt.days

In [107]:
spotify_youtube = pd.concat([spotify_youtube_pt1,spotify_youtube_pt2])

In [108]:
spotify_youtube_dummies= pd.get_dummies(spotify_youtube,columns=['Album_type'], dtype=int)

In [109]:
spotify_youtube_dummies['artist_number'] = spotify_youtube_dummies['Artist'].str.split(',').str.len()
spotify_youtube_dummies['engagement_rate'] = spotify_youtube_dummies.apply(lambda row: (row['Likes'] + row['Comments']) / row['Views'] * 100 if row['Views'] > 0 else 0, axis=1)

In [110]:
columns_drop=['Unnamed: 0', 'Artist', 'Url_spotify', 'Track', 'Album', 'Uri','Url_youtube', 'Title', 'Channel', 'Views', 'Likes','Comments', 'Description', 'Licensed', 'official_video', 'upload_date','Stream']
spotify_youtube_df=spotify_youtube_dummies.drop(columns=columns_drop)

In [111]:
spotify_youtube_limpo=spotify_youtube_df.dropna(subset=['Danceability','Energy','Key','Loudness','Speechiness','Acousticness', 'Instrumentalness','Liveness','Valence','Tempo','Duration_ms','engagement_rate'])

# Preparação dos dados para os experimentos

In [112]:
youtube_30_df=spotify_youtube_limpo[spotify_youtube_limpo['dias_na_plataforma']<31]
youtube_90_df=spotify_youtube_limpo[spotify_youtube_limpo['dias_na_plataforma']<91]
youtube_365_df=spotify_youtube_limpo[spotify_youtube_limpo['dias_na_plataforma']<366]

In [113]:
spotify_youtube_df_366_more = spotify_youtube_limpo.drop(youtube_365_df.index)
spotify_youtube_df_91_365 = youtube_365_df.drop(youtube_90_df.index)
spotify_youtube_df_31_90 = youtube_365_df.drop(youtube_30_df.index)

In [114]:
youtube_30_df_x = youtube_30_df.drop('engagement_rate',axis=1)
youtube_30_df_y = youtube_30_df['engagement_rate']

spotify_youtube_df_31_90_x = spotify_youtube_df_31_90.drop('engagement_rate',axis=1)
spotify_youtube_df_31_90_y = spotify_youtube_df_31_90['engagement_rate']

spotify_youtube_df_91_365_x = spotify_youtube_df_91_365.drop('engagement_rate',axis=1)
spotify_youtube_df_91_365_y = spotify_youtube_df_91_365['engagement_rate']

spotify_youtube_df_366_more_x = spotify_youtube_df_366_more.drop('engagement_rate',axis=1)
spotify_youtube_df_366_more_y = spotify_youtube_df_366_more['engagement_rate']

In [115]:
spotify_youtube_30X_train, spotify_youtube_30X_test, spotify_youtube_30y_train, spotify_youtube_30y_test = train_test_split(youtube_30_df_x, youtube_30_df_y, test_size=0.25, random_state=42)
spotify_youtube_df_31_90X_train, spotify_youtube_df_31_90X_test, spotify_youtube_df_31_90y_train, spotify_youtube_df_31_90y_test = train_test_split(spotify_youtube_df_31_90_x, spotify_youtube_df_31_90_y, test_size=0.25, random_state=42)
spotify_youtube_df_91_365X_train, spotify_youtube_df_91_365X_test, spotify_youtube_df_91_365y_train, spotify_youtube_df_91_365y_test = train_test_split(spotify_youtube_df_91_365_x, spotify_youtube_df_91_365_y, test_size=0.25, random_state=42)

In [116]:
# Dados de treino 
sy_x_train=pd.concat([spotify_youtube_30X_train,spotify_youtube_df_31_90X_train,spotify_youtube_df_91_365X_train,spotify_youtube_df_366_more_x])
sy_y_train=pd.concat([spotify_youtube_30y_train,spotify_youtube_df_31_90y_train,spotify_youtube_df_91_365y_train,spotify_youtube_df_366_more_y])

In [117]:
sy_90_x_train=pd.concat([spotify_youtube_30X_train,spotify_youtube_df_31_90X_train])
sy_90_y_train=pd.concat([spotify_youtube_30y_train,spotify_youtube_df_31_90y_train])

In [118]:
sy_365_x_train=pd.concat([spotify_youtube_30X_train,spotify_youtube_df_31_90X_train,spotify_youtube_df_91_365X_train])
sy_365_y_train=pd.concat([spotify_youtube_30y_train,spotify_youtube_df_31_90y_train,spotify_youtube_df_91_365y_train])

In [119]:
# Para testar o desempenho do modelo com videos com ate 30 dias
sy_30_x_test=spotify_youtube_30X_test
sy_30_y_test=spotify_youtube_30y_test

# Para testar o desempenho do modelo com videos com ate 90 dias
sy_90_x_test=pd.concat([spotify_youtube_30X_test,spotify_youtube_df_31_90X_test])
sy_90_y_test=pd.concat([spotify_youtube_30y_test,spotify_youtube_df_31_90y_test])

# Para testar o desempenho do modelo com videos com ate 1 ano
sy_365_x_test=pd.concat([spotify_youtube_30X_test,spotify_youtube_df_31_90X_test,spotify_youtube_df_91_365X_test])
sy_365_y_test=pd.concat([spotify_youtube_30y_test,spotify_youtube_df_31_90y_test,spotify_youtube_df_91_365y_test])

# Primeira esperimento com os dados

## Regressão com Random Forest

In [120]:
rf = RandomForestRegressor(n_estimators=100, random_state=42) # Random Forest

In [121]:
# Treinamento
rf.fit(sy_x_train,sy_y_train)

RandomForestRegressor(random_state=42)

In [122]:
# Predição
predictions_30_days_rf=rf.predict(sy_30_x_test)# 30 dias
predictions_90_days_rf=rf.predict(sy_90_x_test)# 90 dias
predictions_365_days_rf=rf.predict(sy_365_x_test)# 365 dias

In [123]:
# Erro quadratico medio para 30 dias
mse_30_days = mean_squared_error(sy_30_y_test, predictions_30_days_rf)

# Calculo Desvio padrão
squared_errors_30_days = (sy_30_y_test - predictions_30_days_rf) ** 2
std_mse_30_days = np.std(squared_errors_30_days)

print(f'Erro quadrático médio para músicas com até 30 dias: {mse_30_days:.4f}, desvio padrão: {std_mse_30_days:.4f}')

Erro quadrático médio para músicas com até 30 dias: 3.1562, desvio padrão: 5.0005


In [124]:
# Erro quadratico medio para 90 dias
mse_90_days = mean_squared_error(sy_90_y_test, predictions_90_days_rf)

# Calculo Desvio padrão
squared_errors_90_days = (sy_90_y_test - predictions_90_days_rf) ** 2
std_mse_90_days = np.std(squared_errors_90_days)

print(f'Erro quadrático médio para músicas com até 90 dias: {mse_90_days:.4f}, desvio padrão: {std_mse_90_days:.4f}')

Erro quadrático médio para músicas com até 90 dias: 1.6844, desvio padrão: 10.1863


In [125]:
# Erro quadratico medio para 365 dias
mse_365_days = mean_squared_error(sy_365_y_test, predictions_365_days_rf)

# Calculo Desvio padrão
squared_errors_365_days = (sy_365_y_test - predictions_365_days_rf) ** 2
std_mse_365_days = np.std(squared_errors_365_days)

print(f'Erro quadrático médio para músicas com até 365 dias: {mse_365_days:.4f}, desvio padrão: {std_mse_365_days:.4f}')

Erro quadrático médio para músicas com até 365 dias: 1.3183, desvio padrão: 8.7402


## Regressão com logistic regressor

In [126]:
lin_reg = LinearRegression() # Regressor logistico

In [127]:
# Treinamento
lin_reg.fit(sy_x_train, sy_y_train)

LinearRegression()

In [128]:
# Predição
predictions_30_days_lin=lin_reg.predict(sy_30_x_test)# 30 dias
predictions_90_days_lin=lin_reg.predict(sy_90_x_test)# 90 dias
predictions_365_days_lin=lin_reg.predict(sy_365_x_test)# 365 dias

In [129]:
# Erro quadratico medio para 30 dias
mse_30_days = mean_squared_error(sy_30_y_test, predictions_30_days_lin)

# Calculo Desvio padrão
squared_errors_30_days = (sy_30_y_test - predictions_30_days_lin) ** 2
std_mse_30_days = np.std(squared_errors_30_days)

print(f'Erro quadrático médio para músicas com até 30 dias: {mse_30_days:.4f}, desvio padrão: {std_mse_30_days:.4f}')

Erro quadrático médio para músicas com até 30 dias: 7.0513, desvio padrão: 11.4618


In [130]:
# Erro quadratico medio para 90 dias
mse_90_days = mean_squared_error(sy_90_y_test, predictions_90_days_lin)

# Calculo Desvio padrão
squared_errors_90_days = (sy_90_y_test - predictions_90_days_lin) ** 2
std_mse_90_days = np.std(squared_errors_90_days)

print(f'Erro quadrático médio para músicas com até 90 dias: {mse_90_days:.4f}, desvio padrão: {std_mse_90_days:.4f}')

Erro quadrático médio para músicas com até 90 dias: 3.4343, desvio padrão: 23.2039


In [131]:
# Erro quadratico medio para 365 dias
mse_365_days = mean_squared_error(sy_365_y_test, predictions_365_days_lin)

# Calculo Desvio padrão
squared_errors_365_days = (sy_365_y_test - predictions_365_days_lin) ** 2
std_mse_365_days = np.std(squared_errors_365_days)

print(f'Erro quadrático médio para músicas com até 365 dias: {mse_365_days:.4f}, desvio padrão: {std_mse_365_days:.4f}')

Erro quadrático médio para músicas com até 365 dias: 2.8189, desvio padrão: 19.1513


## Regressão com XGBoost

In [132]:
xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, random_state=42) # xbg para regressão

In [133]:
# Treinamento
xgb_reg.fit(sy_x_train, sy_y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [134]:
# Predição
predictions_30_days_xgb=xgb_reg.predict(sy_30_x_test)# 30 dias
predictions_90_days_xgb=xgb_reg.predict(sy_90_x_test)# 90 dias
predictions_365_days_xgb=xgb_reg.predict(sy_365_x_test)# 365 dias

In [135]:
# Erro quadratico medio para 30 dias
mse_30_days = mean_squared_error(sy_30_y_test, predictions_30_days_xgb)

# Calculo Desvio padrão
squared_errors_30_days = (sy_30_y_test - predictions_30_days_xgb) ** 2
std_mse_30_days = np.std(squared_errors_30_days)

print(f'Erro quadrático médio para músicas com até 30 dias: {mse_30_days:.4f}, desvio padrão: {std_mse_30_days:.4f}')

Erro quadrático médio para músicas com até 30 dias: 3.8066, desvio padrão: 5.9589


In [136]:
# Erro quadratico medio para 90 dias
mse_90_days = mean_squared_error(sy_90_y_test, predictions_90_days_xgb)

# Calculo Desvio padrão
squared_errors_90_days = (sy_90_y_test - predictions_90_days_xgb) ** 2
std_mse_90_days = np.std(squared_errors_90_days)

print(f'Erro quadrático médio para músicas com até 90 dias: {mse_90_days:.4f}, desvio padrão: {std_mse_90_days:.4f}')

Erro quadrático médio para músicas com até 90 dias: 2.0280, desvio padrão: 11.0071


In [137]:
# Erro quadratico medio para 365 dias
mse_365_days = mean_squared_error(sy_365_y_test, predictions_365_days_xgb)

# Calculo Desvio padrão
squared_errors_365_days = (sy_365_y_test - predictions_365_days_xgb) ** 2
std_mse_365_days = np.std(squared_errors_365_days)

print(f'Erro quadrático médio para músicas com até 365 dias: {mse_365_days:.4f}, desvio padrão: {std_mse_365_days:.4f}')

Erro quadrático médio para músicas com até 365 dias: 1.7388, desvio padrão: 9.8049


## Regressão com MLP

In [138]:
mlp_reg = MLPRegressor(random_state=1, max_iter=2000, tol=0.001) # MLP para regressão

In [139]:
# Treinamento
mlp_reg.fit(sy_x_train, sy_y_train)

MLPRegressor(max_iter=2000, random_state=1, tol=0.001)

In [140]:
# Predição
predictions_30_days_mlp=mlp_reg.predict(sy_30_x_test)# 30 dias
predictions_90_days_mlp=mlp_reg.predict(sy_90_x_test)# 90 dias
predictions_365_days_mlp=mlp_reg.predict(sy_365_x_test)# 365 dias

In [141]:
# Erro quadratico medio para 30 dias
mse_30_days = mean_squared_error(sy_30_y_test, predictions_30_days_mlp)

# Calculo Desvio padrão
squared_errors_30_days = (sy_30_y_test - predictions_30_days_mlp) ** 2
std_mse_30_days = np.std(squared_errors_30_days)

print(f'Erro quadrático médio para músicas com até 30 dias: {mse_30_days:.4f}, desvio padrão: {std_mse_30_days:.4f}')

Erro quadrático médio para músicas com até 30 dias: 934.2614, desvio padrão: 558.5439


In [142]:
# Erro quadratico medio para 90 dias
mse_90_days = mean_squared_error(sy_90_y_test, predictions_90_days_mlp)

# Calculo Desvio padrão
squared_errors_90_days = (sy_90_y_test - predictions_90_days_mlp) ** 2
std_mse_90_days = np.std(squared_errors_90_days)

print(f'Erro quadrático médio para músicas com até 90 dias: {mse_90_days:.4f}, desvio padrão: {std_mse_90_days:.4f}')

Erro quadrático médio para músicas com até 90 dias: 1257.1955, desvio padrão: 1513.7960


In [143]:
# Erro quadratico medio para 365 dias
mse_365_days = mean_squared_error(sy_365_y_test, predictions_365_days_mlp)

# Calculo Desvio padrão
squared_errors_365_days = (sy_365_y_test - predictions_365_days_mlp) ** 2
std_mse_365_days = np.std(squared_errors_365_days)

print(f'Erro quadrático médio para músicas com até 365 dias: {mse_365_days:.4f}, desvio padrão: {std_mse_365_days:.4f}')

Erro quadrático médio para músicas com até 365 dias: 1238.4791, desvio padrão: 1415.2241


# Segundo Experimento

Treinado com dados de 30 dias

In [82]:
rf = RandomForestRegressor(n_estimators=100, random_state=42) # Random Forest

In [83]:
# Treinamento
rf.fit(spotify_youtube_30X_train,spotify_youtube_30y_train)

RandomForestRegressor(random_state=42)

In [84]:
# Predição
predictions_30_days_rf=rf.predict(sy_30_x_test)# 30 dias

In [ ]:
# Erro quadratico medio para 30 dias 3.6842, desvio padrão: 6.5005
mse_30_days = mean_squared_error(sy_30_y_test, predictions_30_days_rf)

# Calculo Desvio padrão
squared_errors_30_days = (sy_30_y_test - predictions_30_days_rf) ** 2
std_mse_30_days = np.std(squared_errors_30_days)

print(f'Erro quadrático médio para músicas com até 30 dias: {mse_30_days:.4f}, desvio padrão: {std_mse_30_days:.4f}')

Erro quadrático médio para músicas com até 30 dias: 3.6585, desvio padrão: 6.3363


Treinado com dados de 90 dias

In [88]:
rf = RandomForestRegressor(n_estimators=100, random_state=42) # Random Forest

In [89]:
# Treinamento
rf.fit(sy_90_x_train,sy_90_y_train)

RandomForestRegressor(random_state=42)

In [90]:
# Predição
predictions_30_days_rf=rf.predict(sy_30_x_test)# 30 dias

In [91]:
# Erro quadratico medio para 30 dias 3.6842, desvio padrão: 6.5005
mse_30_days = mean_squared_error(sy_30_y_test, predictions_30_days_rf)

# Calculo Desvio padrão
squared_errors_30_days = (sy_30_y_test - predictions_30_days_rf) ** 2
std_mse_30_days = np.std(squared_errors_30_days)

print(f'Erro quadrático médio para músicas com até 30 dias: {mse_30_days:.4f}, desvio padrão: {std_mse_30_days:.4f}')

Erro quadrático médio para músicas com até 30 dias: 3.5228, desvio padrão: 6.3460


In [92]:
predictions_90_days_rf=rf.predict(sy_90_x_test)# 90 dias

In [95]:
# Erro quadratico medio para 90 dias 3.6842, desvio padrão: 6.5005
mse_90_days = mean_squared_error(sy_90_y_test, predictions_90_days_rf)

# Calculo Desvio padrão
squared_errors_90_days = (sy_90_y_test - predictions_90_days_rf) ** 2
std_mse_90_days = np.std(squared_errors_90_days)

print(f'Erro quadrático médio para músicas com até 90 dias: {mse_90_days:.4f}, desvio padrão: {std_mse_90_days:.4f}')

Erro quadrático médio para músicas com até 90 dias: 2.3043, desvio padrão: 15.1386


Treinando com dados de 365 dias

In [97]:
rf = RandomForestRegressor(n_estimators=100, random_state=42) # Random Forest

In [98]:
# Treinamento
rf.fit(sy_365_x_train,sy_365_y_train)

RandomForestRegressor(random_state=42)

In [99]:
# Predição
predictions_30_days_rf=rf.predict(sy_30_x_test)# 30 dias

In [100]:
# Erro quadratico medio para 30 dias 3.6842, desvio padrão: 6.5005
mse_30_days = mean_squared_error(sy_30_y_test, predictions_30_days_rf)

# Calculo Desvio padrão
squared_errors_30_days = (sy_30_y_test - predictions_30_days_rf) ** 2
std_mse_30_days = np.std(squared_errors_30_days)

print(f'Erro quadrático médio para músicas com até 30 dias: {mse_30_days:.4f}, desvio padrão: {std_mse_30_days:.4f}')

Erro quadrático médio para músicas com até 30 dias: 3.5896, desvio padrão: 6.3719


In [101]:
predictions_90_days_rf=rf.predict(sy_90_x_test)# 90 dias

In [102]:
# Erro quadratico medio para 90 dias 3.6842, desvio padrão: 6.5005
mse_90_days = mean_squared_error(sy_90_y_test, predictions_90_days_rf)

# Calculo Desvio padrão
squared_errors_90_days = (sy_90_y_test - predictions_90_days_rf) ** 2
std_mse_90_days = np.std(squared_errors_90_days)

print(f'Erro quadrático médio para músicas com até 90 dias: {mse_90_days:.4f}, desvio padrão: {std_mse_90_days:.4f}')

Erro quadrático médio para músicas com até 90 dias: 1.3361, desvio padrão: 4.7574


In [103]:
predictions_365_days_rf=rf.predict(sy_365_x_test)# 365 dia

In [104]:
# Erro quadratico medio para 365 dias
mse_365_days = mean_squared_error(sy_365_y_test, predictions_365_days_rf)

# Calculo Desvio padrão
squared_errors_365_days = (sy_365_y_test - predictions_365_days_rf) ** 2
std_mse_365_days = np.std(squared_errors_365_days)

print(f'Erro quadrático médio para músicas com até 365 dias: {mse_365_days:.4f}, desvio padrão: {std_mse_365_days:.4f}')

Erro quadrático médio para músicas com até 365 dias: 1.0005, desvio padrão: 3.9877
